In [15]:
import numpy as np

In [16]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [17]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [18]:
CONSTRAINTS

array([[213,  71],
       [290, 133],
       [191, 111]])

In [19]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [20]:
solution

array([[ True,  True, False, False, False,  True,  True,  True,  True,
         True],
       [False,  True,  True, False,  True,  True,  True, False,  True,
        False],
       [False, False, False, False, False, False,  True,  True,  True,
         True]])

In [21]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.False_

In [22]:
# Check if the solution is valid
all_knapsacks = np.any(solution, axis=0)
np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

np.False_

## TEST PROBLEMS

In [59]:
def fitness_increment(solution, prev_loads, prev_value, i, old_k, new_k):
    loads = prev_loads.copy()
    value = prev_value
    if old_k > 0:
        loads[old_k-1] -= WEIGHTS[i]
        value -= VALUES[i]
    if new_k > 0:
        loads[new_k-1] += WEIGHTS[i]
        value += VALUES[i]
        if np.any(loads[new_k-1] > CONSTRAINTS[new_k-1]):
            return -1, prev_loads, prev_value
    return value, loads, value

In [52]:
def fitness(solution, VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, NUM_DIMENSIONS):
    # Lets compute the total value and the weights for a initial solution

    current_loads = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    current_value = 0

    for i, k in enumerate(solution):
            # k is the knapsack assigned
            if k > 0:
                knapsack_idx = k - 1

                current_loads[knapsack_idx] += WEIGHTS[i]

                # Verify immediatly the constraints
                if np.any(current_loads[knapsack_idx] > CONSTRAINTS[knapsack_idx]):
                    return -1, current_loads, 0

                # Update the values if the constraits are respected
                current_value += VALUES[i]


    return current_value, current_loads, current_value


In [53]:
# HILL CLIMBING
def hill_climb(MAX_ITER=20000, seed=None):
    if seed is not None:
        rng = np.random.default_rng(seed=seed)
    else:
        rng = np.random.default_rng()

    # Starting from a random solution
    solution = rng.integers(0, NUM_KNAPSACKS + 1, size=NUM_ITEMS)

    initial_fit, initial_loads, initial_value = fitness(
        solution, VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, NUM_DIMENSIONS
    )

    best_solution = solution.copy()

    # If the starting solution is not valid (value = -1),
    # hill climbing loop will update it with the first valid one
    best_value = initial_value
    best_loads = initial_loads

    if best_value == -1:
        best_solution = np.zeros(NUM_ITEMS, dtype=int)
        best_loads = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
        best_value = 0

    for _ in range(MAX_ITER):
        new_solution = best_solution.copy()
        new_loads = best_loads.copy()
        new_value = best_value


        i = rng.integers(0, NUM_ITEMS)
        old_k = new_solution[i]
        possible_states = list(range(0, NUM_KNAPSACKS + 1))
        possible_states.remove(old_k)
        new_k = rng.choice(possible_states)
        new_solution[i] = new_k


        # if best_value is 0, the first update computes the fitness of the first run
        fit, loads, value = fitness_increment(new_solution, best_loads, best_value, i, old_k, new_k)

        if fit > best_value:
            best_solution = new_solution
            best_loads = loads
            best_value = value

    return best_solution, best_loads, best_value

In [54]:
def multi_hill_climbing(NUM_RUNS, MAX_ITER_PER_RUN, seed_runs=None):

    #Running Multi-Start Hill Climbing.

    # Inizializing the tracking of the best global solution
    best_solution_global = np.zeros(NUM_ITEMS, dtype=int)
    best_loads_global = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    best_value_global = -1

    print(f"Starting Multi-Start Hill Climbing with {NUM_RUNS} run ({MAX_ITER_PER_RUN} iter/run)...")


    run_rng = np.random.default_rng(seed=seed_runs)

    # Running Hill Climbing for a runs' number desired
    for run in range(1, NUM_RUNS + 1):
        current_seed = run_rng.integers(0, 100000)
        solution, loads, value = hill_climb(MAX_ITER=MAX_ITER_PER_RUN, seed=current_seed)

        # Updating the best global solution
        if value > best_value_global:
            best_value_global = value
            best_solution_global = solution.copy()
            best_loads_global = loads.copy()
            print(f"Run {run}: Best value found: {best_value_global}")

    return best_solution_global, best_loads_global, best_value_global

In [55]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

# Imposing parameters for the Multi-Start
NUM_RUNS = 50
MAX_ITER_PER_RUN = 5000

print("--- STARTING OPTIMIZATION ---")
best_solution, best_loads, best_value = multi_hill_climbing(NUM_RUNS, MAX_ITER_PER_RUN, seed_runs=100)

print("\n--- FINAL RESULT (MSHC) ---")
print("Best value found:", best_value)
for k in range(1, NUM_KNAPSACKS + 1):
    items = [i for i, x in enumerate(best_solution) if x == k]
    print(f"Knapsack {k} ({len(items)} item): {items}")
print("Total weights per knapsack:\n", best_loads)

--- STARTING OPTIMIZATION ---
Starting Multi-Start Hill Climbing with 50 run (5000 iter/run)...
Run 1: Best value found: 223
Run 2: Best value found: 354
Run 5: Best value found: 492
Run 7: Best value found: 970
Run 22: Best value found: 1022
Run 28: Best value found: 1065

--- FINAL RESULT (MSHC) ---
Best value found: 1065
Knapsack 1 (10 item): [0, 2, 4, 7, 9, 11, 14, 16, 18, 19]
Knapsack 2 (6 item): [3, 5, 6, 8, 12, 13]
Knapsack 3 (4 item): [1, 10, 15, 17]
Total weights per knapsack:
 [[598 455]
 [229 427]
 [158 190]]


In [56]:
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000*2, 1000*NUM_ITEMS//NUM_KNAPSACKS, size=NUM_DIMENSIONS)

# Imposing parameters for the Multi-Start
NUM_RUNS = 100
MAX_ITER_PER_RUN = 10000

print("--- STARTING OPTIMIZATION ---")
best_solution, best_loads, best_value = multi_hill_climbing(NUM_RUNS, MAX_ITER_PER_RUN, seed_runs=100)

print("\n--- FINAL RESULT (MSHC) ---")
print("Best value found:", best_value)
for k in range(1, NUM_KNAPSACKS + 1):
    items = [i for i, x in enumerate(best_solution) if x == k]
    print(f"Knapsack {k} ({len(items)} item): {items}")
print("Total weights per knapsack:\n", best_loads)

--- STARTING OPTIMIZATION ---
Starting Multi-Start Hill Climbing with 100 run (10000 iter/run)...
Run 1: Best value found: 6246
Run 5: Best value found: 7316
Run 43: Best value found: 8939

--- FINAL RESULT (MSHC) ---
Best value found: 8939
Knapsack 1 (12 item): [1, 7, 9, 16, 19, 29, 35, 55, 57, 65, 69, 82]
Knapsack 2 (19 item): [2, 3, 17, 23, 27, 31, 36, 40, 41, 42, 45, 47, 53, 59, 60, 67, 79, 80, 84]
Knapsack 3 (11 item): [26, 28, 34, 37, 38, 39, 52, 54, 63, 87, 92]
Knapsack 4 (13 item): [4, 8, 13, 33, 61, 71, 72, 73, 76, 83, 86, 88, 97]
Knapsack 5 (13 item): [6, 10, 25, 43, 51, 58, 66, 74, 75, 89, 90, 91, 95]
Knapsack 6 (12 item): [5, 18, 22, 24, 44, 48, 64, 68, 78, 81, 93, 99]
Knapsack 7 (3 item): [15, 49, 85]
Knapsack 8 (4 item): [21, 62, 70, 96]
Knapsack 9 (6 item): [0, 12, 30, 46, 94, 98]
Knapsack 10 (7 item): [11, 14, 20, 32, 50, 56, 77]
Total weights per knapsack:
 [[2057  696 1383 1715 1871  906  890 2270 2112 1111]
 [2283 1525 2202 1883 1410 3170 1453 1425 2339 2113]
 [1049 

In [58]:
# Problem 3
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000*10, 1000*2*NUM_ITEMS//NUM_KNAPSACKS, size=NUM_DIMENSIONS)

# Imposing parameters for the Multi-Start
NUM_RUNS = 1000
MAX_ITER_PER_RUN = 10000


print("--- STARTING OPTIMIZATION ---")
best_solution, best_loads, best_value = multi_hill_climbing(NUM_RUNS, MAX_ITER_PER_RUN, seed_runs=100)

print("\n--- FINAL RESULT (MSHC) ---")
print("Best value found:", best_value)

print("Total weights per knapsack:\n", best_loads)

--- STARTING OPTIMIZATION ---
Starting Multi-Start Hill Climbing with 1000 run (10000 iter/run)...
Run 1: Best value found: 20420
Run 3: Best value found: 28548
Run 5: Best value found: 28705
Run 20: Best value found: 29321
Run 102: Best value found: 31032
Run 250: Best value found: 31347
Run 263: Best value found: 32974

--- FINAL RESULT (MSHC) ---
Best value found: 32974
Total weights per knapsack:
 [[ 5696  7007  5325 ...  6565  5164  6461]
 [10800  9764  8535 ... 12055 11260 10265]
 [ 4545  4749  5027 ...  6308  4451  5501]
 ...
 [ 6047  4856  7660 ...  5615  5646  6064]
 [ 6780  7744  6911 ...  5886  8054  3604]
 [ 4198  4118  3381 ...  4068  3960  5628]]
